This notebook loads the training data, builds the model and evalutes the preformance. 

## Setup

Installs/Imports

In [2]:
!pip install opencv-python 

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/d9/64/7fdfb9386511cd6805451e012c537073a79a958a58795c4e602e538c388c/opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.2 MB)


In [3]:
import pandas as pd
import numpy as np
import keras
from keras import layers
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

2024-05-13 18:53:23.979305: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-13 18:53:24.213198: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 18:53:25.198853: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Read In Image data

Create data loader function and load image data into data frame

### read_data() definition

*purpose*: to load data in from directory folder and organise metadata

*input/output*: takes in list of names of folders containing image data, reads the image file, resizes the image to 224x224 pixel, converts the image data to a NumPy array and appends it to the data lis 
Appends the labesubfolder name), object ID (extracted from the image file name), sublabel (part of the image file name), and age (also from the image file naes) and appends it to each respective lst.s

In [4]:
def read_data(folder):
    data, labels, objectid, age, sublabel = [], [], [], [], []
    for label in folder:
        path = f"{label}/"
        folder_data = os.listdir(path)
        for image_path in folder_data:
            if image_path.endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
                #print(path + image_path)
                image_path_list = image_path.split('_')
                img = cv2.imread(path + image_path, cv2.IMREAD_UNCHANGED)
                img = cv2.resize(img, (224, 224))
                data.append(np.array(img))
                labels.append(label)
                objectid.append(int(image_path_list[0]))
                sublabel.append(image_path_list[1])
                age.append(image_path_list[2].split('.')[0])

    return data, labels, objectid, age, sublabel

### load data

In [5]:
folder = ['even', 'uneven'] #folders my images are split into
data, labels,  objectid, age, sublabel = read_data(folder) #load data into %notebook

In [9]:
#metadata data frame creation
metadata = pd.DataFrame(
    {
        'labels': labels, # dependent variable, Y
        'objectid': objectid, #unique row label from original data set
        'age': age, #label indicating image is from before or after
        'sublabel': sublabel #silvi_1 label 
    }
) 

## Stacking images

Create functions to assist in stacking task and consolidate metadata with stacked images' infomation

### index_finder()
*purpose*: find index matching criteria provided

*input/output*: Given object id and age return corrisoponding index

In [10]:
def index_finder(objectid, age):
    return metadata.loc[(metadata.objectid == objectid) & (metadata.age == age)].index[0]
    

### combined_image()
*purpose*: to stack image bands of the before and after versions of the same object id

*input/output* given object id, find before and after images in folders and stack the bands

In [11]:
def combine_image(objectid):
    return np.concatenate([data[index_finder(objectid, 'before')], data[index_finder(objectid, 'after')]],axis=2)

### Stack Loop

In [16]:
#create empty lists for stacked images metadata
combined_sublabel = []
combined_labels = []
combined_objectid = []
combined_data = []

objectid_set = set(objectid) # list of unique object ids

#loop to stack all images
for objectid in objectid_set: 
    #combine image
    combined = combine_image(objectid)
    #do not keep duplicates, the following if not keeps duplicates from being added
    if not np.all(combined[:,:,:3] == combined[:,:,3:]):
        combined_objectid.append(objectid)
        index = index_finder(objectid, 'after')
        combined_sublabel.append(sublabel[index])
        combined_labels.append(labels[index])
        combined_data.append(combine_image(objectid))
    

In [ ]:
len(combined_data) #any changes?

## Train/Test Split

Create new dataframe and split data into train adn test groups

In [17]:
#Combined/stacked images data frame (duplicate before/after images removed)
df = pd.DataFrame(
    {
        'data': combined_data, 
        'label': combined_labels, 
        'objectid': combined_objectid
    }
)

train, test = train_test_split(df, test_size=0.2, random_state=12)

print("Train Size: ", train.shape[0])
print("Test Size: ", test.shape[0])

Train Size:  1357
Test Size:  340


## Prepare the data

Put infomarion into correct format to load into model

In [33]:
# Model data parameters
num_classes = 2
input_shape = (224, 224, 6)
# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = (train.data, train.label), (test.data, test.label)

# Extract values from DataFrame
x_train_values = np.array([np.array(img) for img in x_train])
x_test_values = np.array([np.array(img) for img in x_test])

# Scale images to the [0, 1] range
x_train_values = x_train_values.astype("float32") / 255
x_test_values = x_test_values.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("x_train shape:", x_train_values.shape)
print(x_train_values.shape[0], "train samples")
print(x_test_values.shape[0], "test samples")

# Make sure images have shape (224, 224, 6)
x_train_values = np.expand_dims(x_train_values, -1)
x_test_values = np.expand_dims(x_test_values, -1)

from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and transform labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train_encoded, num_classes)
y_test = keras.utils.to_categorical(y_test_encoded, num_classes)

x_train shape: (1357, 224, 224, 6)
1357 train samples
340 test samples
(1357, 1)
(1357, 224, 224, 6, 1)


## Build the model

The model consists of three pairs of convolutional and max pooling layers, followed by a flattening layer, a dropout layer for regularization, and a dense output layer for classification

In [27]:
model = keras.Sequential( #initialize sequential model
    [
        keras.Input(shape=input_shape), #set input layer sixe (224, 224, 6)
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"), #adds a 2D convolutional layer w/ 32 kernels, each of size 3x3
        layers.MaxPooling2D(pool_size=(2, 2)), #adds a max pooling layer with a pool size of 2x2, makes next layers input smaller 
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"), #adds a 2D convolutional layer w/ 64 kernels, each of size 3x3
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"), 
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(), #transforms the input data into a 1D array
        layers.Dropout(0.5), #adds dropout layer w/ rate = .5
        #(Dropout: regularization technique to prevent overfitting by randomly setting a fraction of input units to 0 during training.)
        layers.Dense(num_classes, activation="softmax"), #adds a fully connected (dense) layer with num_classes neurons
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      1760      
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 32)        0

2024-05-13 20:23:32.601628: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43348 MB memory:  -> device: 0, name: NVIDIA L40, pci bus id: 0000:e1:00.0, compute capability: 8.9


## Train the model
Set batch size and number of epochs and fit model with training data

In [113]:
batch_size = 32
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train_values, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
39/39 [==============================] - 3s 24ms/step - loss: 0.6415 - accuracy: 0.6568 - val_loss: 0.4262 - val_accuracy: 0.8088
Epoch 2/15
39/39 [==============================] - 1s 19ms/step - loss: 0.3997 - accuracy: 0.8313 - val_loss: 0.3845 - val_accuracy: 0.8235
Epoch 3/15
39/39 [==============================] - 1s 28ms/step - loss: 0.3843 - accuracy: 0.8387 - val_loss: 0.5532 - val_accuracy: 0.7426
Epoch 4/15
39/39 [==============================] - 1s 29ms/step - loss: 0.3821 - accuracy: 0.8436 - val_loss: 0.3581 - val_accuracy: 0.8529
Epoch 5/15
39/39 [==============================] - 1s 19ms/step - loss: 0.3207 - accuracy: 0.8600 - val_loss: 0.3775 - val_accuracy: 0.8088
Epoch 6/15
39/39 [==============================] - 1s 19ms/step - loss: 0.3038 - accuracy: 0.8755 - val_loss: 0.3430 - val_accuracy: 0.8456
Epoch 7/15
39/39 [==============================] - 1s 22ms/step - loss: 0.2804 - accuracy: 0.8829 - val_loss: 0.3374 - val_accuracy: 0.8676
Epoch 8/15
39

## Evaluate the trained model

Use test data to gauge models preformance

In [114]:
score = model.evaluate(x_test_values, y_test, verbose=0)
print("Test loss:", score[0]) #lower the better
print("Test accuracy:", score[1]) #higher the better, less than .5 accracy is worse than just guessing

Test loss: 0.4968973994255066
Test accuracy: 0.8441176414489746
